In [1]:
from sklearn import metrics
import xgboost
from xgboost import XGBRegressor
#!pip install catboost
#import catboost
#from catboost import CatBoostRegressor
#!pip install lightgbm
#import lightgbm
#from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetRegressor

import torch

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp28x2r0ks
  JVM stdout: /tmp/tmp28x2r0ks/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp28x2r0ks/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_unknownUser_5cdien
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


## AutoML

In [9]:
train = h2o.import_file('/content/syn_sages.csv')

import time
import itertools
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [60]:
train_, valid = train.split_frame(ratios = [.85])
for cols in ['Dip', 'Slice', 'Block', 'SAGES']:
  train_[cols] = train_[cols].asfactor()
  valid[cols] = valid[cols].asfactor()

X = train_.drop(['Withdrawal load rise', "Slice"], axis=1)

gbm = H2OGradientBoostingEstimator(ntrees = 1000,
    stopping_metric = "rmse",
    nfolds=6)

gbm.train(x=X.columns, y="Withdrawal load rise", training_frame=train_, validation_frame=valid)

print(gbm)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1623519704518_12


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,1000.0,1000.0,109484.0,2.0,4.0,2.733,3.0,5.0,3.958




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.0017161615279630996
RMSE: 0.04142657996942421
MAE: 0.03211167279411765
RMSLE: 0.00033452727346971323
Mean Residual Deviance: 0.0017161615279630996

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 1101.8332544407333
RMSE: 33.19387374864123
MAE: 27.091041905750807
RMSLE: 0.29226721763113445
Mean Residual Deviance: 1101.8332544407333

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 1716.9801251643614
RMSE: 41.43645888784853
MAE: 33.29561506998827
RMSLE: 0.342551292345115
Mean Residual Deviance: 1716.9801251643614

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid,cv_6_valid
0,mae,33.00777,4.5298247,32.44625,33.429516,31.639511,39.813374,34.824577,25.893402
1,mean_residual_deviance,1668.6664,519.2462,1559.1938,1468.8729,1725.288,2322.8833,2094.1174,841.64325
2,mse,1668.6664,519.2462,1559.1938,1468.8729,1725.288,2322.8833,2094.1174,841.64325
3,r2,-0.92653555,1.043393,-1.06723,0.0036473295,-1.5160449,-0.9975949,-2.4396894,0.4576986
4,residual_deviance,1668.6664,519.2462,1559.1938,1468.8729,1725.288,2322.8833,2094.1174,841.64325
5,rmse,40.386333,6.7180724,39.48663,38.32588,41.536587,48.1963,45.761528,29.01109
6,rmsle,0.32168743,0.078758374,0.3623569,0.26290566,0.3184927,0.39113864,0.39739627,0.19783428



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2021-06-12 18:30:42,17.359 sec,0.0,33.880242,28.539792,1147.870819,23.237074,17.133690,539.961623
1,,2021-06-12 18:30:42,17.362 sec,1.0,32.775432,27.569106,1074.228926,22.658550,16.147438,513.409885
2,,2021-06-12 18:30:42,17.365 sec,2.0,31.834215,26.737781,1013.417230,22.403530,15.472688,501.918149
3,,2021-06-12 18:30:42,17.367 sec,3.0,30.678757,25.496638,941.186156,22.683064,15.605192,514.521390
4,,2021-06-12 18:30:42,17.369 sec,4.0,29.934727,24.927931,896.087871,22.548256,15.691490,508.423847
5,,2021-06-12 18:30:42,17.372 sec,5.0,29.064303,24.192603,844.733729,22.611519,15.984728,511.280779
6,,2021-06-12 18:30:42,17.374 sec,6.0,28.339670,23.565830,803.136867,22.722953,16.451477,516.332600
7,,2021-06-12 18:30:42,17.377 sec,7.0,27.604331,22.916725,761.999092,22.993136,17.180240,528.684305
8,,2021-06-12 18:30:42,17.379 sec,8.0,27.029576,22.457244,730.597983,23.015426,17.373432,529.709848
9,,2021-06-12 18:30:42,17.382 sec,9.0,26.209858,21.644706,686.956635,23.113574,18.053498,534.237283



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip,69480.867188,1.000000,0.225505
1,Conv rise mm,44311.539062,0.637752,0.143816
2,Set date,38532.164062,0.554572,0.125059
3,Face m,26622.474609,0.383163,0.086405
4,Dip,25117.955078,0.361509,0.081522
5,Rise load,20597.224609,0.296445,0.066850
6,withdrawal time min,19598.460938,0.282070,0.063608
7,Dip load,17053.759766,0.245445,0.055349
8,Roof exposure,15694.945312,0.225889,0.050939
9,Conv dip mm,10294.395508,0.148162,0.033411


In [42]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

interaction_pairs = [("Dip load", "Rise load"),
                    ("Withdrawal load dip", "Withdrawal load rise"),
                     ("Conv rise mm", "Conv dip mm")]

glm2 = H2OGeneralizedLinearEstimator(nfolds=5,
                                    alpha=0.25,
                                    interaction_pairs=interaction_pairs,
                                    keep_cross_validation_models=True,
                                    stopping_metric='rmse',
                                    family='poisson',
                                    seed=1
                                    )

X = train_.drop(['Withdrawal load rise'], axis=1)

glm2.train(x = X.columns, y = 'Withdrawal load rise', training_frame = train_, validation_frame = valid)

print(glm2)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1623519704518_9


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.25, lambda = 10.412 )",49,4,2,py_51_sid_ae98




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 256.3436398396377
RMSE: 16.010735143635277
MAE: 12.981024517984713
RMSLE: 0.13420523166915632
R^2: 0.736329724458286
Mean Residual Deviance: 2.092387561323033
Null degrees of freedom: 52
Residual degrees of freedom: 48
Null deviance: 396.5401663642061
Residual deviance: 110.89654075012074
AIC: 475.0577780952768

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 637.6079796994983
RMSE: 25.250900572048877
MAE: 23.103668318242175
RMSLE: 0.2270367846085238
R^2: 0.47285763206912723
Mean Residual Deviance: 5.466815625920341
Null degrees of freedom: 8
Residual degrees of freedom: 4
Null deviance: 106.71356106491692
Residual deviance: 49.20134063328307
AIC: 118.12207011755012

Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-06-12 18:21:26,0.000 sec,0,396.540166,7.481890,,,,,,,,
1,,2021-06-12 18:21:26,0.002 sec,1,109.674833,2.777723,,,,,,,,
2,,2021-06-12 18:21:26,0.020 sec,2,110.896541,2.777572,16.0107,2.09239,12.981,0.73633,25.2509,5.46682,23.1037,0.472858



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip_Withdrawal load rise,0.165375,1.000000,0.750426
1,Conv rise mm,0.030252,0.182927,0.137273
2,Withdrawal load dip,0.013507,0.081677,0.061293
3,withdrawal time min,0.011241,0.067972,0.051008
4,Slice.46,0.000000,0.000000,0.000000
5,Slice.47,0.000000,0.000000,0.000000
6,Slice.48,0.000000,0.000000,0.000000
7,Slice.49,0.000000,0.000000,0.000000
8,Slice.50,0.000000,0.000000,0.000000
9,Slice.51,0.000000,0.000000,0.000000



See the whole table with table.as_data_frame()



In [62]:
rf = SVR()
x = sage[[col for col in sage.columns if col not in ['Withdrawal load rise', 'Set date', 'Withdraw date', 'Set']]]
y = sage['Withdrawal load rise']
#xt, xtt, yt, ytt = train_test_split(x, y, test_size=0.15)
rf.fit(xt, yt)
print('rmse', np.sqrt(metrics.mean_squared_error(ytt, rf.predict(xtt))))

rmse 22.396533209399138


In [3]:
!pip install table-evaluator

In [4]:
!pip install ctgan

In [5]:
import pandas as pd
sage = pd.read_csv('/content/sages.csv')

In [6]:
from ctgan import CTGANSynthesizer

# Identifies all the discrete columns

discrete_columns = [col for col in sage.columns]

# Initiates the CTGANSynthesizer and call its fit method to pass in the table
 
ctgan = CTGANSynthesizer(epochs=10)
ctgan.fit(sage, discrete_columns)

In [7]:
synthetic_data = ctgan.sample(1000)

In [11]:
synthetic_data.head()

,Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
0,28.0,5,59,427,1/20/2018,97,124,11/21/2017,140,108,50,3,11,26,910,0,-1,-1
1,28.5,6,67,399,12/29/2017,103,66,11/14/2017,181,125,6,8,10,17,1840,31,1,1
2,31.0,9,51,447,4/9/2017,144,143,9/26/2017,143,73,5,9,8,94,1530,7,-1,1
3,31.0,9,64,561,12/27/2017,69,108,8/19/2017,151,143,1,5,4,13,1720,18,-1,1
4,31.0,11,52,549,8/2/2018,144,118,11/12/2017,131,114,2,5,5,125,1730,25,1,-1


In [12]:
synthetic_data.shape

(500, 18)

In [8]:
synthetic_data.to_csv('/content/syn_sages.csv', index=False)

In [14]:
sage.columns

Index(['Level', 'Dip', 'Slice', 'Face m', 'Set date', 'Dip load', 'Rise load',
       'Withdraw date', 'Withdrawal load dip', 'Withdrawal load rise',
       'Conv dip mm', 'Conv rise mm', 'Slice extraction days',
       'withdrawal time min', 'Roof exposure', 'Blasting holes', 'Block',
       'SAGES'],
      dtype='object')

In [19]:
train[541:545, :]

Level,Dip,Slice,Face m,Set date,Dip load,Rise load,Withdraw date,Withdrawal load dip,Withdrawal load rise,Conv dip mm,Conv rise mm,Slice extraction days,withdrawal time min,Roof exposure,Blasting holes,Block,SAGES
28,5,59,381,2017-06-11 00:00:00,134,35,2017-11-21 00:00:00,92,156,14,3,8,22,810,5,1,1
28.5,9,66,339,2017-09-29 00:00:00,70,131,2017-11-20 00:00:00,173,156,8,6,1,9,1110,22,1,-1
31,5,54,459,2017-08-30 00:00:00,94,70,2018-02-07 00:00:00,129,136,0,1,6,10,2350,27,1,-1
28.5,9,58,435,2017-08-12 00:00:00,98,111,2017-12-14 00:00:00,181,96,4,8,9,147,1230,7,-1,1


In [22]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
train_, valid = train.split_frame(ratios = [.85])
for cols in ['Dip', 'Slice', 'Block', 'SAGES']:
  train_[cols] = train_[cols].asfactor()
  valid[cols] = valid[cols].asfactor()
interaction_pairs = [("Dip load", "Rise load"),
                    ("Withdrawal load dip", "Withdrawal load rise"),
                     ("Conv rise mm", "Conv dip mm"),
                      ('Dip load', 'Rise load')]

glm2 = H2OGeneralizedLinearEstimator(nfolds=5,
                                    alpha=0,
                                    interaction_pairs=interaction_pairs,
                                    keep_cross_validation_models=True,
                                    stopping_metric='rmse',
                                    family='poisson',
                                    seed=12
                                    )

X = train_.drop(['Withdrawal load rise'], axis=1)

glm2.train(x = X.columns, y = 'Withdrawal load rise', training_frame = train_, validation_frame = valid)

print(glm2)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1623533504698_8


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,Ridge ( lambda = 0.02201 ),50,50,4,py_85_sid_8c15




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 109.57756870050638
RMSE: 10.467930487947767
MAE: 7.167223643069243
RMSLE: 0.08132699525426647
R^2: 0.8990879763669275
Mean Residual Deviance: 0.7803981066058896
Null degrees of freedom: 854
Residual degrees of freedom: 804
Null deviance: 7178.38205331529
Residual deviance: 667.2403811480356
AIC: 6484.708658369526

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 106.59212673835985
RMSE: 10.324346310462461
MAE: 6.998780806507256
RMSLE: 0.0685679500200198
R^2: 0.8987616188733389
Mean Residual Deviance: 0.6673928078314353
Null degrees of freedom: 144
Residual degrees of freedom: 94
Null deviance: 1150.5972796456144
Residual deviance: 96.77195713555811
AIC: 1169.0882675664807

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 140.0075055826319
RMSE: 11.832476730703167
MAE: 7.725202349983816
RMSLE: 0.08734237344741004
R^2: 0.8710644808083171
Mean Residual Deviance: 0.93697

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,7.7292833,0.49963456,7.8846464,7.3011813,7.5929255,7.3469515,8.520712
1,mean_residual_deviance,0.93758315,0.15443727,1.0656475,0.7308828,1.0195677,0.8143546,1.0574633
2,mse,139.49443,53.453346,222.93222,93.55801,121.69537,99.26135,160.02519
3,null_deviance,1436.3754,101.57688,1522.0116,1283.6183,1383.1749,1480.1929,1512.8795
4,r2,0.8728237,0.04317639,0.8025068,0.9044187,0.8864418,0.9078871,0.862864
5,residual_deviance,160.22256,26.327263,189.68524,123.519196,168.22868,144.14076,175.53891
6,rmse,11.649625,2.1739025,14.930915,9.672539,11.031563,9.962999,12.650106
7,rmsle,0.08717615,0.008503224,0.07973768,0.07905868,0.09928778,0.08612597,0.09167065



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-06-12 21:41:09,0.000 sec,0,7178.382053,8.395768,,,,,,,,
1,,2021-06-12 21:41:09,0.003 sec,1,801.946915,0.940165,,,,,,,,
2,,2021-06-12 21:41:09,0.004 sec,2,667.314081,0.782749,,,,,,,,
3,,2021-06-12 21:41:09,0.007 sec,3,667.240381,0.782664,,,,,,,,
4,,2021-06-12 21:41:09,0.008 sec,4,667.240381,0.782664,10.4679,0.780398,7.16722,0.899088,10.3243,0.667393,6.99878,0.898762



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip_Withdrawal load rise,0.354998,1.000000,0.355250
1,Withdrawal load dip,0.268545,0.756470,0.268736
2,Slice.50,0.044721,0.125975,0.044753
3,Slice.62,0.032904,0.092688,0.032927
4,Slice.47,0.031108,0.087628,0.031130
5,Slice.68,0.030376,0.085567,0.030398
6,Slice.52,0.024816,0.069905,0.024834
7,Slice.51,0.020745,0.058438,0.020760
8,Slice.66,0.017168,0.048360,0.017180
9,Slice.67,0.016649,0.046900,0.016661



See the whole table with table.as_data_frame()



In [24]:
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [23]:
model_path = h2o.save_model(model=glm2, path="/content/Withdraw_rise", force=True)

In [49]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(sage, synthetic_data)
for col in sage.columns:
  print(table_evaluator.evaluate(target_col=col))


Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1_real  f1_fake  jaccard_similarity
index                                                                    
LogisticRegression_real_testset       0.8462   0.4615              0.1818
LogisticRegression_fake_testset       0.3846   0.2308              0.1818
RandomForestClassifier_real_testset   1.0000   0.3846              0.2381
RandomForestClassifier_fake_testset   0.3077   0.3077              0.2381
DecisionTreeClassifier_real_testset   1.0000   0.5385              0.3684
DecisionTreeClassifier_fake_testset   0.3077   0.2308              0.3000
MLPClassifier_real_testset            0.8462   0.2308              0.1818
MLPClassifier_fake_testset            0.3846   0.5385              0.3684

Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicat

/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))
/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))



Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    4.0784
nearest neighbor std                     0.5275

Results:
                                                Result
Basic statistics                                0.9774
Correlation column correlations                 0.7781
Mean Correlation between fake and real columns  0.9462
1 - MAPE Estimator results                         nan
Similarity Score                                   nan
{'Basic statistics': 0.9774262231278188, 'Correlation column correlations': 0.778140385017866, 'Mean Correlation between fake and real columns': 0.9462021048114119, '1 - MAPE Estimator results': nan, 'Similarity Score': nan}

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1

/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))



Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    4.0784
nearest neighbor std                     0.5275

Results:
                                                Result
Basic statistics                                0.9774
Correlation column correlations                 0.7781
Mean Correlation between fake and real columns  0.9462
1 - MAPE Estimator results                         nan
Similarity Score                                   nan
{'Basic statistics': 0.9774262231278188, 'Correlation column correlations': 0.778140385017866, 'Mean Correlation between fake and real columns': 0.9462021048114119, '1 - MAPE Estimator results': nan, 'Similarity Score': nan}

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1

...


Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    4.0784
nearest neighbor std                     0.5275

Results:
                                                Result
Basic statistics                                0.9774
Correlation column correlations                 0.7781
Mean Correlation between fake and real columns  0.9462
1 - MAPE Estimator results                         nan
Similarity Score                                   nan
{'Basic statistics': 0.9774262231278188, 'Correlation column correlations': 0.778140385017866, 'Mean Correlation between fake and real columns': 0.9462021048114119, '1 - MAPE Estimator results': nan, 'Similarity Score': nan}

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1

/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))
/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))



Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    4.0784
nearest neighbor std                     0.5275

Results:
                                                Result
Basic statistics                                0.9774
Correlation column correlations                 0.7781
Mean Correlation between fake and real columns  0.9462
1 - MAPE Estimator results                         nan
Similarity Score                                   nan
{'Basic statistics': 0.9774262231278188, 'Correlation column correlations': 0.778140385017866, 'Mean Correlation between fake and real columns': 0.9462021048114119, '1 - MAPE Estimator results': nan, 'Similarity Score': nan}

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1

/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: divide by zero encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))
/usr/local/lib/python3.7/dist-packages/table_evaluator/metrics.py:39: RuntimeWarning: invalid value encountered in true_divide
  return np.mean(np.abs((y_true - y_pred) / y_true))



Miscellaneous results:
                                         Result
Column Correlation Distance RMSE         0.2962
Column Correlation distance MAE          0.2197
Duplicate rows between sets (real/fake)  (0, 0)
nearest neighbor mean                    4.0784
nearest neighbor std                     0.5275

Results:
                                                Result
Basic statistics                                0.9774
Correlation column correlations                 0.7781
Mean Correlation between fake and real columns  0.9462
1 - MAPE Estimator results                         nan
Similarity Score                                   nan
{'Basic statistics': 0.9774262231278188, 'Correlation column correlations': 0.778140385017866, 'Mean Correlation between fake and real columns': 0.9462021048114119, '1 - MAPE Estimator results': nan, 'Similarity Score': nan}

Correlation metric: pearsonr

Classifier F1-scores and their Jaccard similarities:
                                     f1

In [30]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
train_, valid = train.split_frame(ratios = [.85])
for cols in ['Dip', 'Slice', 'Block', 'SAGES']:
  train_[cols] = train_[cols].asfactor()
  valid[cols] = valid[cols].asfactor()
interaction_pairs = [("Dip load", "Rise load"),
                    ("Withdrawal load dip", "Withdrawal load rise"),
                     ("Conv rise mm", "Conv dip mm"),
                      ('Dip load', 'Rise load')]

glm3 = H2OGeneralizedLinearEstimator(nfolds=5,
                                    alpha=0,
                                    interaction_pairs=interaction_pairs,
                                    keep_cross_validation_models=True,
                                    stopping_metric='rmse',
                                    family='poisson',
                                    seed=12
                                    )

X = train_.drop(['Withdrawal load dip'], axis=1)

glm3.train(x = X.columns, y = 'Withdrawal load dip', training_frame = train_, validation_frame = valid)

print(glm3)

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:200: RuntimeWarning: Stopping metric is ignored for _stopping_rounds=0.
  warnings.warn(mesg["message"], RuntimeWarning)


glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1623533504698_9


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,Ridge ( lambda = 0.02822 ),50,50,4,py_96_sid_8c15




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 119.70461301107402
RMSE: 10.94096033312771
MAE: 8.000912629661133
RMSLE: 0.1015482861168423
R^2: 0.9187826215196108
Mean Residual Deviance: 0.9713648180765219
Null degrees of freedom: 851
Residual degrees of freedom: 801
Null deviance: 9789.060884828421
Residual deviance: 827.6028250011966
AIC: 6651.68384249046

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 100.1257244082575
RMSE: 10.006284245825595
MAE: 7.271668409285983
RMSLE: 0.11572076456620867
R^2: 0.9228438952522819
Mean Residual Deviance: 1.0363985795022919
Null degrees of freedom: 147
Residual degrees of freedom: 97
Null deviance: 1547.0542577302697
Residual deviance: 153.38698976633918
AIC: 1248.3461085456279

ModelMetricsRegressionGLM: glm
** Reported on cross-validation data. **

MSE: 136.42015446532272
RMSE: 11.679903872263791
MAE: 8.479812688473055
RMSLE: 0.10599359373721452
R^2: 0.9074414340528524
Mean Residual Deviance: 1.07994

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,8.485912,0.30610973,8.182498,8.363938,8.5822735,8.330692,8.97016
1,mean_residual_deviance,1.0824308,0.1434225,0.9969799,0.9446453,1.1753834,1.00817,1.2869754
2,mse,136.5304,16.755207,124.2684,113.26264,149.0648,150.23024,145.82587
3,null_deviance,1963.4126,185.98792,2142.574,2115.5576,1915.5819,1679.1051,1964.2445
4,r2,0.9055573,0.019131208,0.9199265,0.92957205,0.89789397,0.8815303,0.8988636
5,residual_deviance,184.02328,20.520807,176.46544,159.64505,193.93828,176.42975,213.63791
6,rmse,11.666391,0.72946465,11.147573,10.642492,12.209209,12.2568445,12.075838
7,rmsle,0.10573017,0.01130993,0.10126923,0.09839693,0.113833524,0.09405697,0.12109421



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
0,,2021-06-12 21:48:05,0.000 sec,0,9789.060885,11.489508,,,,,,,,
1,,2021-06-12 21:48:05,0.004 sec,1,1047.627726,1.232262,,,,,,,,
2,,2021-06-12 21:48:05,0.005 sec,2,827.855362,0.974762,,,,,,,,
3,,2021-06-12 21:48:05,0.007 sec,3,827.602878,0.974489,,,,,,,,
4,,2021-06-12 21:48:05,0.010 sec,4,827.602825,0.974489,10.941,0.971365,8.00091,0.918783,10.0063,1.0364,7.27167,0.922844



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Withdrawal load dip_Withdrawal load rise,0.379024,1.000000,0.383626
1,Withdrawal load rise,0.266706,0.703664,0.269944
2,Slice.50,0.043454,0.114647,0.043982
3,Slice.62,0.034214,0.090268,0.034629
4,Slice.61,0.024809,0.065454,0.025110
5,Slice.63,0.023762,0.062692,0.024050
6,Slice.48,0.023295,0.061461,0.023578
7,Slice.68,0.019032,0.050214,0.019263
8,Slice.64,0.018413,0.048580,0.018637
9,Slice.66,0.013724,0.036210,0.013891



See the whole table with table.as_data_frame()



In [31]:
model_path = h2o.save_model(model=glm3, path="/content/Withdraw_dip", force=True)